In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\mayuo\\OneDrive\\Documents\\Machine Learning by Abraham\\MLOps wine'

#### code your entity 

In [4]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

#### Code config manager

In [5]:

from src.MLOps_wine.constants import *
from src.MLOps_wine.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

#### code your component

In [7]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [29]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,true, predicted):
        accuracy = accuracy_score(true, predicted)
        precision = precision_score(true, predicted, average='weighted')  # Update here
        recall = recall_score(true, predicted, average='weighted')  # Update here
        f1 = f1_score(true, predicted, average='weighted')  # Update here
        return accuracy, precision, recall, f1

    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        label_mapping = {'low': 0, 'medium': 1, 'high': 2}
        test_data[self.config.target_column] = test_data[self.config.target_column].map(label_mapping)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        predicted_qualities = model.predict(test_x)

        (accuracy, precision, recall, f1) = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1':f1}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [30]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e


[2024-04-30 12:28:22,512: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-30 12:28:22,514: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-30 12:28:22,517: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-30 12:28:22,519: INFO: common: created directory at: artifacts]
[2024-04-30 12:28:22,520: INFO: common: created directory at: artifacts/model_evaluation]
[2024-04-30 12:28:22,541: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\MLOps wine\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
